In [1]:
import pandas as pd

from matplotlib import pyplot as plt
import matplotlib.dates as mdates

import altair as alt
from altair import datum

In [2]:
def review_date_import_clean(product_name):
    df_review = pd.read_csv('data/{}.csv'.format(product_name))
    df_review = df_review[df_review['verified_purchase'] == 1]
    df_review['review_date_key'] = pd.to_datetime(df_review['review_date'])
    df_review_weekly = df_review.groupby(pd.Grouper(key='review_date_key', freq='W-SUN', label='left', closed='left'))['review_rating']\
                        .agg(['mean', 'count'])\
                        .reset_index()
    #df_review_weekly['review_date_key'] = df_review_weekly['review_date_key'] + pd.to_timedelta(1, unit='d')
    df_review_weekly.rename(columns={'mean':'average_rating', 'count':'review_count'}, inplace=True)
    
    return df_review_weekly

In [3]:
def google_data_import_clean(product_name):
    df_gg = pd.read_excel('data/Google Trend.xlsx', sheet_name = product_name)
    df_gg['week_key'] = pd.to_datetime(df_gg['Week'])
    
    df_gg.rename(columns = {df_gg.columns[1]:'trend_index'}, inplace = True)
    df_gg_filter = df_gg[df_gg['trend_index'].cumsum() > 0]
    
    return df_gg_filter

In [4]:
def data_import_clean(product_name):
    
    df_review_clean = review_date_import_clean(product_name)
    df_gg_clean = google_data_import_clean(product_name)
    
    df = df_review_clean.merge(df_gg_clean, left_on = 'review_date_key', right_on='week_key', how='right')
    df['product_name'] = product_name
    df = df.sort_values('week_key').reset_index().drop('index', axis=1)
    
    return df

In [5]:
def visualize_review_google_trend(df):
    
    fig = plt.figure(figsize=(20,10))
    ax = fig.add_subplot(111)

    lns1 = ax.plot(df['week_key'], df['review_count'], '-b', label = 'Amazon Review Count')
    ax2 = ax.twinx()
    lns2 = ax2.plot(df['week_key'], df['trend_index'],'-r', label = 'Google Trend Index')

    lns = lns1+lns2
    labs = [l.get_label() for l in lns]
    ax.legend(lns, labs, loc=0, fontsize=16)

    ax.set_xlabel("Week")
    ax.set_ylabel("Review Count")
    ax2.set_ylabel("Trend Index")

    fig.tight_layout()
    plt.show()

In [6]:
def visualize_correlation(df):
    corr_lst = [(i, df['review_count'].corr(df['trend_index'].shift(i))) for i in range(len(df)//2)]
    plt.bar([i[0] for i in corr_lst], [i[1] for i in corr_lst])
    print(max(corr_lst,key=lambda item:item[1]))

In [7]:
def calculate_correlation(df):
    corr_lst = [(i, df['review_count'].corr(df['trend_index'].shift(i))) for i in range(len(df)//2)]
    df_corr = pd.DataFrame(corr_lst, columns=['lag', 'corr'])
    df_corr['product_name'] = df['product_name'][0]
    df_corr['total_reviews'] = df['review_count'].sum()
    return df_corr

In [8]:
product_list = ['AirPods Pro', 'Elite 75t', 'Free', 'Galaxy Buds Plus', 'NC700', 'Soundcore Liberty 2 Pro', 'Soundcore Liberty Air 2', 'Soundcore Life Q20', 'WF-1000XM3']
df_corr = pd.DataFrame(columns=['lag', 'corr', 'product_name', 'total_reviews'])

for product in product_list:
    df = data_import_clean(product)
    df_corr = pd.concat([df_corr, calculate_correlation(df)])

In [9]:
alt.Chart(df_corr).mark_bar(size=10).encode(
    x=alt.X('lag'),
    y=alt.Y('corr', 
            #scale=alt.Scale(domain=[0,1])
           ),
    facet=alt.Facet(
        'product_name',
        columns=1,
        sort=['NC700', 'AirPods Pro', 'WF-1000XM3', 'Elite 75t', 'Soundcore Liberty 2 Pro', 'Galaxy Buds Plus', 'Soundcore Liberty Air 2', 'Soundcore Life Q20', 'Free']
    )
).transform_filter(
    datum.corr > 0
).resolve_scale(x='independent')

alt.Chart(...)